In [2]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kbojja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [95]:
# pip install pdfminer.six
# from pdfminer.high_level import extract_text
# text = extract_text('gan_prediction.pdf')
# text

'Available online at www.sciencedirect.com\nAvailable online at www.sciencedirect.com\n\nProcedia Computer Science 00 (2019) 000–000\nProcedia Computer Science 00 (2019) 000–000\n\nProcedia Computer Science 147 (2019) 400–406\n\nwww.elsevier.com/locate/procedia\nwww.elsevier.com/locate/procedia\n\n2018 International Conference on Identiﬁcation, Information and Knowledge\n2018 International Conference on Identiﬁcation, Information and Knowledge\n\nin the Internet of Things, IIKI 2018\nin the Internet of Things, IIKI 2018\n\nStock Market Prediction Based on Generative Adversarial Network\nStock Market Prediction Based on Generative Adversarial Network\n\nKang Zhanga, Guoqiang Zhonga,∗, Junyu Donga, Shengke Wanga, Yong Wanga\nKang Zhanga, Guoqiang Zhonga,∗, Junyu Donga, Shengke Wanga, Yong Wanga\n\naDepartment of Computer Science and Technology, Ocean University of China, Qingdao, 266100, China\naDepartment of Computer Science and Technology, Ocean University of China, Qingdao, 266100, Ch

# read file, extract text and form tokens

In [3]:
from pdfminer.high_level import extract_text

def read_article(file_name):
    text = extract_text(file_name) # extracts text from given pdf file
    article = text.split(". ")
    sentences = []

    for sentence in article:
        print(sentence)
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

In [4]:
def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
 
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)

In [5]:
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

In [6]:
def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
      summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Offcourse, output the summarize texr
    print("Summarized Text: \n", ". ".join(summarize_text))

In [7]:
generate_summary( "seq2sql.pdf", 2)

7
1
0
2

 

v
o
N
9

 

 
 
]
L
C
.
s
c
[
 
 

7
v
3
0
1
0
0

.

9
0
7
1
:
v
i
X
r
a

SEQ2SQL: GENERATING STRUCTURED QUERIES
FROM NATURAL LANGUAGE USING REINFORCEMENT
LEARNING

Victor Zhong, Caiming Xiong, Richard Socher
Salesforce Research
Palo Alto, CA
{vzhong,cxiong,rsocher}@salesforce.com

ABSTRACT

Relational databases store a signiﬁcant amount of the worlds data
However, ac-
cessing this data currently requires users to understand a query language such as
SQL
We propose Seq2SQL, a deep neural network for translating natural lan-
guage questions to corresponding SQL queries
Our model uses rewards from in-
the-loop query execution over the database to learn a policy to generate the query,
which contains unordered parts that are less suitable for optimization via cross en-
tropy loss
Moreover, Seq2SQL leverages the structure of SQL to prune the space
of generated queries and signiﬁcantly simplify the generation problem
In addition
to the model, we release WikiSQL, a dataset of 80654

Indexes of top ranked_sentence order are  [(0.009775413283603927, ['By', 'leveraging', 'the', 'inherent', 'structure', 'of\nSQL', 'queries', 'and', 'applying', 'policy', 'gradient', 'methods', 'using', 'reward', 'signals', 'from', 'live', 'query', 'execution,\nSeq2SQL', 'achieves', 'state-of-the-art', 'performance', 'on', 'WikiSQL,', 'obtaining', '59.4%', 'execution', 'accuracy.\n\n2', 'MODEL\n\nThe', 'WikiSQL', 'task', 'is', 'to', 'generate', 'a', 'SQL', 'query', 'from', 'a', 'natural', 'language', 'question', 'and', 'table', 'schema.\nOur', 'baseline', 'model', 'is', 'the', 'attentional', 'sequence', 'to', 'sequence', 'neural', 'semantic', 'parser', 'proposed', 'by', 'Dong\n&', 'Lapata', '(2016)', 'that', 'achieves', 'state-of-the-art', 'performance', 'on', 'a', 'host', 'of', 'semantic', 'parsing', 'datasets\nwithout', 'using', 'hand-engineered', 'grammar']), (0.007833918231926527, ['With', 'this', 'augmented', 'input', 'sequence,', 'the', 'pointer', 'network\ncan', 'produce', 'the',